# Knowledge Distillation for COVID Tweet Sentiment Analysis

This notebook implements knowledge distillation to compress a BERTweet sentiment analysis model into a smaller DistilRoBERTa student model.

## Distillation Setup:
- **Teacher Model**: `finiteautomata/bertweet-base-sentiment-analysis` (frozen)
- **Student Model**: `distilroberta-base` with classification head
- **Loss Function**: Combined hard target (CrossEntropy) + soft target (KL Divergence) loss
- **Temperature**: T ≈ 2.0 for soft target computation
- **Alpha**: α ≈ 0.5 for loss weighting


## Environment Setup


In [1]:
# IMPORTANT: Run this cell first to set up CPU-only environment
import os

# Set environment variables to force CPU usage and disable MPS
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

print("Environment configured for CPU-only execution")
print("Please restart the kernel and run all cells from the beginning")


Environment configured for CPU-only execution
Please restart the kernel and run all cells from the beginning


## Installations


In [2]:
%pip install -q evaluate
%pip install -q emoji==0.6.0
%pip install -q torch
%pip install -q transformers
%pip install -q accelerate
%pip install -q wandb


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Imports


In [3]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch import nn
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, cohen_kappa_score
from transformers import (
    TrainingArguments,
    Trainer,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainerCallback,
    EarlyStoppingCallback
)
import evaluate
import wandb
import time
from datetime import datetime


/Users/taltzafrir/At_Bay/paz/deep/deep_learning_sentiment/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Device Setup


In [4]:
# Smart device selection for cross-platform GPU support
def get_optimal_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using CUDA GPU: {torch.cuda.get_device_name()}")
        return device
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using Apple Silicon MPS")
        return device
    else:
        device = torch.device("cpu")
        print("Using CPU")
        return device

device = get_optimal_device()

Using Apple Silicon MPS


In [5]:
# W&B login
print('Logging to wandb.ai account')
wandb.login(key="6dd13a6018f089606e418d323dd8b502f31bca4e")


Logging to wandb.ai account


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/taltzafrir/.netrc
wandb: Currently logged in as: ttzafrir (at-bay-data-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Data Loading and Preprocessing


In [6]:
# Read raw data
train = pd.read_csv("OOT_train.csv", encoding='latin-1')
val = pd.read_csv("OOT_val.csv", encoding='latin-1')
test = pd.read_csv("OOT_test.csv", encoding='latin-1')


### Label Encoding


In [7]:
# Encoding the labels numerically from Sentiment
ordinal_mapping = {
    'Extremely Negative': 0,
    'Negative': 1,
    'Neutral': 2,
    'Positive': 3,
    'Extremely Positive': 4
}

# Map to ordinal labels
train["ordinal_label_id"] = train["Sentiment"].map(ordinal_mapping)
val["ordinal_label_id"] = val["Sentiment"].map(ordinal_mapping)
test["ordinal_label_id"] = test["Sentiment"].map(ordinal_mapping)

# Define mapping between label id and sentiment for later use
ordinal_label2id = ordinal_mapping
ordinal_id2label = {v: k for k, v in ordinal_mapping.items()}


### Augmented Input Building


In [8]:
# Function to build the input string from multiple columns
def build_augmented_input(row):
    parts = []

    if pd.notna(row.get('clean_tweet')):
        parts.append(f"{row['clean_tweet']}")

    if pd.notna(row.get('Location_standardized')) and row['Location_standardized'].lower() != 'unknown':
        parts.append(f"{row['Location_standardized']}")

    if pd.notna(row.get('TweetAt')):
        parts.append(f"{row['TweetAt']}")

    return " | ".join(parts)

# Apply to the DataFrames
train['model_input'] = train.apply(build_augmented_input, axis=1)
val['model_input'] = val.apply(build_augmented_input, axis=1)
test['model_input'] = test.apply(build_augmented_input, axis=1)

# Create new DataFrames with only what's needed for modeling
formatted_train = train[['model_input', 'ordinal_label_id']].copy()
formatted_val = val[['model_input', 'ordinal_label_id']].copy()
formatted_test = test[['model_input', 'ordinal_label_id']].copy()


### Dataset Balancing


In [9]:
def balance_dataset(df, target_samples_per_class=5000):
    """Balance dataset by undersampling"""
    balanced_dfs = []

    print("Original class distribution:")
    print(df['ordinal_label_id'].value_counts().sort_index())

    for class_id in range(5):
        class_data = df[df['ordinal_label_id'] == class_id]

        if len(class_data) > target_samples_per_class:
            class_data = class_data.sample(n=target_samples_per_class, random_state=42)
            print(f"Class {class_id}: {len(class_data)} samples (undersampled)")
        else:
            print(f"Class {class_id}: {len(class_data)} samples (kept all)")

        balanced_dfs.append(class_data)

    balanced_df = pd.concat(balanced_dfs, ignore_index=True).sample(frac=1, random_state=42)

    print(f"Balanced dataset: {len(balanced_df)} total samples")
    print("New distribution:")
    print(balanced_df['ordinal_label_id'].value_counts().sort_index())

    return balanced_df

# Apply balancing to training data
formatted_train = balance_dataset(formatted_train, target_samples_per_class=5000)


Original class distribution:
ordinal_label_id
0     5175
1     9230
2     6784
3    10140
4     5845
Name: count, dtype: int64
Class 0: 5000 samples (undersampled)
Class 1: 5000 samples (undersampled)
Class 2: 5000 samples (undersampled)
Class 3: 5000 samples (undersampled)
Class 4: 5000 samples (undersampled)
Balanced dataset: 25000 total samples
New distribution:
ordinal_label_id
0    5000
1    5000
2    5000
3    5000
4    5000
Name: count, dtype: int64


## Model Setup

### Teacher Model (BERTweet) - Frozen
### Student Model (DistilRoBERTa) - Trainable


In [10]:
# Teacher model configuration
MODEL_TYPE = 'roberta'
teacher_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
teacher_model_path = "./HuggingFace/roberta/best_roberta_model_so_far"
teacher_model_file = "model_roberta.pt"

# Student model configuration
student_model_name = "distilroberta-base"

print(f"Teacher Model: {teacher_model_name}")
print(f"Student Model: {student_model_name}")

Teacher Model: cardiffnlp/twitter-roberta-base-sentiment
Student Model: distilroberta-base


### Load Teacher Model


In [11]:
# Load the best teacher model from hyperparameter tuning
print("Loading teacher model from best checkpoint...")
teacher_model = torch.load(os.path.join(teacher_model_path, teacher_model_file), map_location=device, weights_only=False)
teacher_model.eval()  # Set to evaluation mode
teacher_model.to(device)

# Freeze teacher model parameters
for param in teacher_model.parameters():
    param.requires_grad = False

print("Teacher model loaded and frozen!")

# Load teacher tokenizer
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)


Loading teacher model from best checkpoint...
Teacher model loaded and frozen!


### Initialize Student Model


In [12]:
# Initialize student model with classification head
student_model = AutoModelForSequenceClassification.from_pretrained(
    student_model_name,
    num_labels=5,
    id2label=ordinal_id2label,
    label2id=ordinal_label2id
)
student_model.to(device)

# Load student tokenizer
student_tokenizer = AutoTokenizer.from_pretrained(student_model_name)

print(f"Student model initialized with {sum(p.numel() for p in student_model.parameters() if p.requires_grad):,} trainable parameters")
print(f"Teacher model has {sum(p.numel() for p in teacher_model.parameters()):,} total parameters (frozen)")


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Student model initialized with 82,122,245 trainable parameters
Teacher model has 124,649,477 total parameters (frozen)


## Custom Dataset for Knowledge Distillation


In [13]:
class DistillationDataset(Dataset):
    """Custom dataset for knowledge distillation that stores raw text"""
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __getitem__(self, idx):
        # Return raw text and label - tokenization will happen in the collate function
        return {
            'text': self.texts[idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.labels)


def create_distillation_collate_fn(student_tokenizer, teacher_tokenizer, device, max_length=128):
    """Create a collate function that tokenizes for both student and teacher"""
    def collate_fn(batch):
        texts = [item['text'] for item in batch]
        labels = torch.stack([item['labels'] for item in batch])
        
        # Tokenize for student
        student_encodings = student_tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors='pt'
        )
        
        # Tokenize for teacher
        teacher_encodings = teacher_tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors='pt'
        )
        
        # Move all tensors to the correct device
        return {
            'student_input_ids': student_encodings['input_ids'].to(device),
            'student_attention_mask': student_encodings['attention_mask'].to(device),
            'teacher_input_ids': teacher_encodings['input_ids'].to(device),
            'teacher_attention_mask': teacher_encodings['attention_mask'].to(device),
            'labels': labels.to(device)
        }
    
    return collate_fn

### Create Datasets


In [14]:
# Convert labels to lists
train_texts = formatted_train['model_input'].tolist()
val_texts = formatted_val['model_input'].tolist()
test_texts = formatted_test['model_input'].tolist()

train_labels = formatted_train['ordinal_label_id'].tolist()
val_labels = formatted_val['ordinal_label_id'].tolist()
test_labels = formatted_test['ordinal_label_id'].tolist()

# Create distillation datasets
train_dataset = DistillationDataset(train_texts, train_labels)
val_dataset = DistillationDataset(val_texts, val_labels)
test_dataset = DistillationDataset(test_texts, test_labels)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")


Train dataset size: 25000
Validation dataset size: 4357
Test dataset size: 3424


## Metrics Computation


In [15]:
def compute_detailed_metrics(eval_pred):
    """Enhanced metrics using HuggingFace Evaluate library"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # Load HuggingFace metrics (cached after first load)
    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")

    # Compute standard classification metrics
    results = {}

    # Basic metrics
    results.update(accuracy_metric.compute(predictions=predictions, references=labels))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average='macro'))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average='weighted'))
    results.update(precision_metric.compute(predictions=predictions, references=labels, average='macro'))
    results.update(recall_metric.compute(predictions=predictions, references=labels, average='macro'))

    # Per-class F1 scores
    f1_per_class = f1_score(labels, predictions, average=None)
    for i, class_name in enumerate(['extremely_negative', 'negative', 'neutral', 'positive', 'extremely_positive']):
        results[f'f1_{class_name}'] = f1_per_class[i]

        # Per-class precision and recall
        precision_per_class = precision_score(labels, predictions, average=None, zero_division=0)
        recall_per_class = recall_score(labels, predictions, average=None, zero_division=0)
        results[f'precision_{class_name}'] = precision_per_class[i]
        results[f'recall_{class_name}'] = recall_per_class[i]

        # Per-class accuracy
        class_mask = (labels == i)
        if class_mask.sum() > 0:
            results[f'accuracy_{class_name}'] = accuracy_score(labels[class_mask], predictions[class_mask])
        else:
            results[f'accuracy_{class_name}'] = 0.0

    # Custom ordinal metrics
    results['mae'] = np.mean(np.abs(predictions - labels))
    results['adjacent_accuracy'] = np.sum(np.abs(predictions - labels) <= 1) / len(labels)

    # Quadratic Weighted Kappa
    try:
        qwk = cohen_kappa_score(labels, predictions, weights='quadratic')
        results['quadratic_weighted_kappa'] = qwk
    except:
        results['quadratic_weighted_kappa'] = 0.0

    return results


## Knowledge Distillation Components


### Distillation Hyperparameters


In [16]:
# Knowledge distillation hyperparameters
TEMPERATURE = 2.0  # Temperature for soft targets
ALPHA = 0.5  # Weight for hard target loss (1-alpha for soft target loss)

print(f"Distillation Temperature: {TEMPERATURE}")
print(f"Alpha (hard loss weight): {ALPHA}")
print(f"Soft loss weight: {1-ALPHA}")


Distillation Temperature: 2.0
Alpha (hard loss weight): 0.5
Soft loss weight: 0.5


### Custom Distillation Trainer


In [17]:
class DistillationTrainer(Trainer):
    """Custom trainer for knowledge distillation"""
    
    def __init__(self, teacher_model, temperature, alpha, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        self.temperature = temperature
        self.alpha = alpha
        
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Compute combined knowledge distillation loss
        """
        # Check if we're in distillation mode (training) or regular mode (evaluation)
        if 'student_input_ids' in inputs and inputs['student_input_ids'] is not None:
            # Distillation mode - use both teacher and student
            # Get student predictions
            student_outputs = model(
                input_ids=inputs.get('student_input_ids'),
                attention_mask=inputs.get('student_attention_mask')
            )
            student_logits = student_outputs.logits
            
            # Get teacher predictions (no gradient)
            with torch.no_grad():
                teacher_outputs = self.teacher_model(
                    input_ids=inputs.get('teacher_input_ids'),
                    attention_mask=inputs.get('teacher_attention_mask')
                )
                teacher_logits = teacher_outputs.logits
            
            # Get labels
            labels = inputs.get('labels')
            
            # Hard target loss (CrossEntropy)
            loss_fct = nn.CrossEntropyLoss()
            hard_loss = loss_fct(student_logits, labels)
            
            # Soft target loss (KL Divergence)
            student_log_probs = F.log_softmax(student_logits / self.temperature, dim=-1)
            teacher_probs = F.softmax(teacher_logits / self.temperature, dim=-1)
            
            # KL Divergence loss
            soft_loss = F.kl_div(student_log_probs, teacher_probs, reduction='batchmean')
            soft_loss = soft_loss * (self.temperature ** 2)
            
            # Combined loss
            loss = self.alpha * hard_loss + (1 - self.alpha) * soft_loss
            
            return (loss, student_outputs) if return_outputs else loss
        
        else:
            # Regular mode - use standard loss computation for evaluation
            student_outputs = model(
                input_ids=inputs.get('input_ids'),
                attention_mask=inputs.get('attention_mask'),
                labels=inputs.get('labels')
            )
            
            return (student_outputs.loss, student_outputs) if return_outputs else student_outputs.loss
        
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        """
        Override prediction step to use student inputs only
        """
        # Check if we have the expected distillation inputs
        if 'student_input_ids' in inputs and inputs['student_input_ids'] is not None:
            # During training/evaluation with distillation data
            student_inputs = {
                'input_ids': inputs.get('student_input_ids'),
                'attention_mask': inputs.get('student_attention_mask'),
                'labels': inputs.get('labels')
            }
        else:
            # Fallback for regular evaluation (when using SimpleDataset)
            student_inputs = {
                'input_ids': inputs.get('input_ids'),
                'attention_mask': inputs.get('attention_mask'),
                'labels': inputs.get('labels')
            }
        
        # Call parent's prediction_step with student inputs
        return super().prediction_step(model, student_inputs, prediction_loss_only, ignore_keys)

## Training Setup


### Metrics Logger Callback


In [18]:
class DistillationMetricsLogger(TrainerCallback):
    """Callback to log detailed metrics during distillation"""
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None or not wandb.run:
            return
        
        # Only log when we have evaluation metrics
        if 'eval_loss' in logs:
            current_epoch = int(state.epoch) if state.epoch else 0
            
            # Get training loss from state history
            train_loss = 0
            if state.log_history:
                for log_entry in reversed(state.log_history):
                    if 'loss' in log_entry:
                        train_loss = log_entry['loss']
                        break
            
            # Detailed metrics
            detailed_metrics = {
                "Epoch": current_epoch,
                "Train Loss": train_loss,
                "Validation Loss": logs.get('eval_loss', 0),
                "Validation Accuracy": logs.get('eval_accuracy', 0),
                "Validation F1": logs.get('eval_f1', 0),
                "Validation MAE": logs.get('eval_mae', 0),
                "Validation QWK": logs.get('eval_quadratic_weighted_kappa', 0),
                "Learning_Rate": logs.get('learning_rate', args.learning_rate),
            }
            
            # Log to WandB
            wandb.log(detailed_metrics)
            
            # Print progress
            print(f"Epoch {current_epoch}: "
                  f"Train Loss: {train_loss:.4f}, "
                  f"Val Loss: {logs.get('eval_loss', 0):.4f}, "
                  f"Val F1: {logs.get('eval_f1', 0):.4f}, "
                  f"QWK: {logs.get('eval_quadratic_weighted_kappa', 0):.4f}")


### Training Configuration


In [19]:
# Initialize W&B for tracking
wandb.init(
    project="covid-tweet-sentiment-distillation",
    name=f"{MODEL_TYPE}-to-distilroberta",
    config={
        "model_type": MODEL_TYPE,
        "teacher_model": teacher_model_name,
        "student_model": student_model_name,
        "temperature": TEMPERATURE,
        "alpha": ALPHA,
        "num_epochs": 10,
        "batch_size": 32,
        "learning_rate": 3e-5,
    }
)

In [20]:
# Training arguments with GPU optimizations
training_args = TrainingArguments(
    output_dir="./distilled_model",
    num_train_epochs=10,
    per_device_train_batch_size=16 if device.type == "cpu" else 32,  # Smaller batch for CPU
    per_device_eval_batch_size=32 if device.type == "cpu" else 64,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_quadratic_weighted_kappa",
    greater_is_better=True,
    save_total_limit=2,
    remove_unused_columns=False,
    label_names=["labels"],
    report_to="wandb",
    fp16=device.type == "cuda",  # Enable FP16 only for CUDA
    dataloader_num_workers=0,
    dataloader_pin_memory=device.type != "cpu",
)

### Create Distillation Trainer


In [21]:
# Create custom collate function
collate_fn = collate_fn = create_distillation_collate_fn(student_tokenizer, teacher_tokenizer, device)

# Create distillation trainer
trainer = DistillationTrainer(
    teacher_model=teacher_model,
    temperature=TEMPERATURE,
    alpha=ALPHA,
    model=student_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_detailed_metrics,
    data_collator=collate_fn,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=2),
        DistillationMetricsLogger(),
    ],
)

# GPU memory optimization
if device.type in ["cuda", "mps"]:
    # Enable gradient checkpointing for memory efficiency
    student_model.gradient_checkpointing_enable()
    
    if device.type == "cuda":
        # CUDA-specific optimizations
        torch.backends.cudnn.benchmark = True
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("Distillation trainer created successfully!")

Distillation trainer created successfully!


## Training with Knowledge Distillation


In [22]:
# Train the student model with knowledge distillation
print("Starting knowledge distillation training...")
trainer.train()

Starting knowledge distillation training...


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,F1 Extremely Negative,Precision Extremely Negative,Recall Extremely Negative,Accuracy Extremely Negative,F1 Negative,Precision Negative,Recall Negative,Accuracy Negative,F1 Neutral,Precision Neutral,Recall Neutral,Accuracy Neutral,F1 Positive,Precision Positive,Recall Positive,Accuracy Positive,F1 Extremely Positive,Precision Extremely Positive,Recall Extremely Positive,Accuracy Extremely Positive,Mae,Adjacent Accuracy,Quadratic Weighted Kappa
1,0.955800,0.774704,0.704384,0.701395,0.724734,0.706781,0.669082,0.814706,0.567623,0.567623,0.677950,0.607979,0.766120,0.766120,0.783908,0.801410,0.767154,0.767154,0.631670,0.706796,0.570980,0.570980,0.768190,0.692777,0.862025,0.862025,0.373881,0.931834,0.830954
2,0.660300,0.743583,0.729401,0.722631,0.729979,0.758834,0.725766,0.609179,0.897541,0.897541,0.605541,0.763727,0.501639,0.501639,0.850114,0.864959,0.835771,0.835771,0.679612,0.735832,0.631373,0.631373,0.782284,0.676199,0.927848,0.927848,0.338536,0.941244,0.862074
3,0.500800,0.564406,0.811797,0.812561,0.815623,0.814478,0.801164,0.760589,0.846311,0.846311,0.753904,0.743100,0.765027,0.765027,0.864560,0.867497,0.861642,0.861642,0.802011,0.790999,0.813333,0.813333,0.846049,0.915929,0.786076,0.786076,0.238008,0.954556,0.892980
4,0.391700,0.572941,0.816846,0.815952,0.816534,0.826372,0.819939,0.814141,0.825820,0.825820,0.771320,0.766739,0.775956,0.775956,0.870203,0.873160,0.867267,0.867267,0.784152,0.827526,0.745098,0.745098,0.855457,0.801105,0.917722,0.917722,0.226762,0.959376,0.903685
5,0.320900,0.752399,0.765664,0.762996,0.759187,0.793921,0.746732,0.620924,0.936475,0.936475,0.676593,0.743455,0.620765,0.620765,0.838968,0.855140,0.823397,0.823397,0.741612,0.834314,0.667451,0.667451,0.822134,0.742100,0.921519,0.921519,0.284599,0.957769,0.887736
6,0.263000,0.587084,0.827863,0.827709,0.827812,0.834648,0.835644,0.808429,0.864754,0.864754,0.786344,0.792453,0.780328,0.780328,0.859099,0.871528,0.847019,0.847019,0.810597,0.817384,0.803922,0.803922,0.863014,0.849265,0.877215,0.877215,0.210466,0.966261,0.909745
7,0.230600,0.730180,0.794354,0.791570,0.790945,0.815442,0.807116,0.743103,0.883197,0.883197,0.746479,0.806084,0.695082,0.695082,0.848652,0.830819,0.867267,0.867267,0.758859,0.845043,0.688627,0.688627,0.822750,0.729677,0.943038,0.943038,0.238237,0.971081,0.909533
8,0.195200,0.701074,0.808813,0.806980,0.808315,0.825939,0.836435,0.801126,0.875000,0.875000,0.779176,0.817527,0.744262,0.744262,0.849833,0.840484,0.859393,0.859393,0.769684,0.830909,0.716863,0.716863,0.832957,0.751527,0.934177,0.934177,0.223319,0.970852,0.913190
9,0.146100,0.745446,0.799403,0.797358,0.793488,0.821795,0.811439,0.719493,0.930328,0.930328,0.741599,0.789149,0.699454,0.699454,0.849858,0.856164,0.843645,0.843645,0.772554,0.839779,0.715294,0.715294,0.834194,0.762854,0.920253,0.920253,0.237549,0.967868,0.907128
10,0.140000,0.729537,0.807895,0.805848,0.804034,0.826150,0.822642,0.762238,0.893443,0.893443,0.764912,0.822642,0.714754,0.714754,0.852260,0.835676,0.869516,0.869516,0.779232,0.843693,0.723922,0.723922,0.833617,0.755922,0.929114,0.929114,0.222860,0.973147,0.914032


Epoch 1: Train Loss: 0.9558, Val Loss: 0.7747, Val F1: 0.7014, QWK: 0.8310
Epoch 2: Train Loss: 0.6603, Val Loss: 0.7436, Val F1: 0.7226, QWK: 0.8621
Epoch 3: Train Loss: 0.5008, Val Loss: 0.5644, Val F1: 0.8126, QWK: 0.8930
Epoch 4: Train Loss: 0.3917, Val Loss: 0.5729, Val F1: 0.8160, QWK: 0.9037
Epoch 5: Train Loss: 0.3209, Val Loss: 0.7524, Val F1: 0.7630, QWK: 0.8877
Epoch 6: Train Loss: 0.2630, Val Loss: 0.5871, Val F1: 0.8277, QWK: 0.9097
Epoch 7: Train Loss: 0.2306, Val Loss: 0.7302, Val F1: 0.7916, QWK: 0.9095
Epoch 8: Train Loss: 0.1952, Val Loss: 0.7011, Val F1: 0.8070, QWK: 0.9132
Epoch 9: Train Loss: 0.1461, Val Loss: 0.7454, Val F1: 0.7974, QWK: 0.9071
Epoch 10: Train Loss: 0.1400, Val Loss: 0.7295, Val F1: 0.8058, QWK: 0.9140


TrainOutput(global_step=7820, training_loss=0.4122857236496323, metrics={'train_runtime': 4311.2458, 'train_samples_per_second': 57.988, 'train_steps_per_second': 1.814, 'total_flos': 0.0, 'train_loss': 0.4122857236496323, 'epoch': 10.0})

## Evaluation and Comparison


### Model Size Comparison


In [23]:
def get_model_size(model):
    """Calculate model size in MB"""
    param_size = 0
    buffer_size = 0
    
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    
    size_mb = (param_size + buffer_size) / (1024 * 1024)
    return size_mb

# Compare model sizes
teacher_size = get_model_size(teacher_model)
student_size = get_model_size(student_model)
compression_ratio = teacher_size / student_size

print(f"\nModel Size Comparison:")
print(f"Teacher Model Size: {teacher_size:.2f} MB")
print(f"Student Model Size: {student_size:.2f} MB")
print(f"Compression Ratio: {compression_ratio:.2f}x")
print(f"Size Reduction: {(1 - student_size/teacher_size) * 100:.1f}%")



Model Size Comparison:
Teacher Model Size: 475.51 MB
Student Model Size: 313.28 MB
Compression Ratio: 1.52x
Size Reduction: 34.1%


### Inference Speed Comparison


In [24]:
def measure_inference_time(model, tokenizer, sample_text, num_runs=50):
    """Measure average inference time for a model"""
    model.eval()
    times = []
    
    # Warm-up runs
    for _ in range(5):
        inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            _ = model(**inputs)
    
    # Actual timing runs
    for _ in range(num_runs):
        inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, padding=True, max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        start_time = time.time()
        with torch.no_grad():
            _ = model(**inputs)
        end_time = time.time()
        
        times.append(end_time - start_time)
    
    return np.mean(times), np.std(times)

# Test inference speed
sample_tweet = "COVID-19 vaccines have been crucial in reducing hospitalizations and saving lives worldwide."

print("Measuring inference speed...")
teacher_time, teacher_std = measure_inference_time(teacher_model, teacher_tokenizer, sample_tweet)
student_time, student_std = measure_inference_time(student_model, student_tokenizer, sample_tweet)

speedup = teacher_time / student_time

print(f"\nInference Speed Comparison:")
print(f"Teacher Model: {teacher_time*1000:.2f} ± {teacher_std*1000:.2f} ms")
print(f"Student Model: {student_time*1000:.2f} ± {student_std*1000:.2f} ms")
print(f"Speedup: {speedup:.2f}x")

Measuring inference speed...

Inference Speed Comparison:
Teacher Model: 15.36 ± 0.27 ms
Student Model: 8.66 ± 0.16 ms
Speedup: 1.77x


### Performance Evaluation


In [25]:
def evaluate_model(model, tokenizer, dataset, model_name):
    """Evaluate a model on a given dataset"""
    model.eval()
    
    # Create regular dataset for single-model evaluation
    eval_encodings = tokenizer(
        [dataset[i]['text'] for i in range(len(dataset))],
        truncation=True,
        padding=False,
        max_length=128,
        add_special_tokens=True,
        return_attention_mask=True,
        return_token_type_ids=False
    )
    
    class SimpleDataset(Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels
        
        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item
        
        def __len__(self):
            return len(self.labels)
    
    eval_dataset = SimpleDataset(eval_encodings, [dataset[i]['labels'].item() for i in range(len(dataset))])
    
    # Create trainer for evaluation
    eval_trainer = Trainer(
        model=model,
        args=TrainingArguments(
            output_dir="./temp",
            per_device_eval_batch_size=64,
            remove_unused_columns=False,
            dataloader_num_workers=0,
        ),
        tokenizer=tokenizer,
        compute_metrics=compute_detailed_metrics,
    )
    
    # Evaluate
    results = eval_trainer.evaluate(eval_dataset)
    
    # Format results with model name prefix
    formatted_results = {}
    for key, value in results.items():
        new_key = f"{model_name}_{key}"
        formatted_results[new_key] = value
    
    return results, formatted_results

# Evaluate both models on validation set
print("Evaluating Teacher Model on Validation Set...")
val_results_teacher, val_formatted_teacher = evaluate_model(teacher_model, teacher_tokenizer, val_dataset, "teacher")

print("\nEvaluating Student Model on Validation Set...")
val_results_student, val_formatted_student = evaluate_model(student_model, student_tokenizer, val_dataset, "student")

# Evaluate both models on test set
print("\nEvaluating Teacher Model on Test Set...")
test_results_teacher, test_formatted_teacher = evaluate_model(teacher_model, teacher_tokenizer, test_dataset, "teacher")

print("\nEvaluating Student Model on Test Set...")
test_results_student, test_formatted_student = evaluate_model(student_model, student_tokenizer, test_dataset, "student")

# Log to wandb
wandb.log({
    **val_formatted_teacher,
    **val_formatted_student,
    **test_formatted_teacher,
    **test_formatted_student
})


Evaluating Teacher Model on Validation Set...


/var/folders/qg/7c5v880d4_lg0yc07pfjklsh0000gn/T/ipykernel_35732/41261118.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(



Evaluating Student Model on Validation Set...


/var/folders/qg/7c5v880d4_lg0yc07pfjklsh0000gn/T/ipykernel_35732/41261118.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(



Evaluating Teacher Model on Test Set...


/var/folders/qg/7c5v880d4_lg0yc07pfjklsh0000gn/T/ipykernel_35732/41261118.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(



Evaluating Student Model on Test Set...


/var/folders/qg/7c5v880d4_lg0yc07pfjklsh0000gn/T/ipykernel_35732/41261118.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(


## Comprehensive Results Summary


In [26]:
print("="*80)
print("KNOWLEDGE DISTILLATION RESULTS SUMMARY")
print("="*80)

print(f"\n📊 MODEL COMPRESSION:")
print(f"Teacher Model Size: {teacher_size:.2f} MB")
print(f"Student Model Size: {student_size:.2f} MB")
print(f"Compression Ratio: {compression_ratio:.2f}x")
print(f"Size Reduction: {(1 - student_size/teacher_size) * 100:.1f}%")

print(f"\n⚡ INFERENCE SPEED:")
print(f"Teacher Model: {teacher_time*1000:.2f} ms")
print(f"Student Model: {student_time*1000:.2f} ms")
print(f"Speedup: {speedup:.2f}x")

print(f"\n🎯 VALIDATION SET PERFORMANCE:")
print(f"                      Teacher     Student     Difference")
print(f"Accuracy:             {val_results_teacher['eval_accuracy']:.4f}      {val_results_student['eval_accuracy']:.4f}      {val_results_student['eval_accuracy'] - val_results_teacher['eval_accuracy']:+.4f}")
print(f"F1-Score:             {val_results_teacher['eval_f1']:.4f}      {val_results_student['eval_f1']:.4f}      {val_results_student['eval_f1'] - val_results_teacher['eval_f1']:+.4f}")
print(f"QWK:                  {val_results_teacher['eval_quadratic_weighted_kappa']:.4f}      {val_results_student['eval_quadratic_weighted_kappa']:.4f}      {val_results_student['eval_quadratic_weighted_kappa'] - val_results_teacher['eval_quadratic_weighted_kappa']:+.4f}")
print(f"MAE:                  {val_results_teacher['eval_mae']:.4f}      {val_results_student['eval_mae']:.4f}      {val_results_student['eval_mae'] - val_results_teacher['eval_mae']:+.4f}")

print(f"\n🧪 TEST SET PERFORMANCE:")
print(f"                      Teacher     Student     Difference")
print(f"Accuracy:             {test_results_teacher['eval_accuracy']:.4f}      {test_results_student['eval_accuracy']:.4f}      {test_results_student['eval_accuracy'] - test_results_teacher['eval_accuracy']:+.4f}")
print(f"F1-Score:             {test_results_teacher['eval_f1']:.4f}      {test_results_student['eval_f1']:.4f}      {test_results_student['eval_f1'] - test_results_teacher['eval_f1']:+.4f}")
print(f"QWK:                  {test_results_teacher['eval_quadratic_weighted_kappa']:.4f}      {test_results_student['eval_quadratic_weighted_kappa']:.4f}      {test_results_student['eval_quadratic_weighted_kappa'] - test_results_teacher['eval_quadratic_weighted_kappa']:+.4f}")
print(f"MAE:                  {test_results_teacher['eval_mae']:.4f}      {test_results_student['eval_mae']:.4f}      {test_results_student['eval_mae'] - test_results_teacher['eval_mae']:+.4f}")

# Calculate performance retention
acc_retention = (test_results_student['eval_accuracy'] / test_results_teacher['eval_accuracy']) * 100
f1_retention = (test_results_student['eval_f1'] / test_results_teacher['eval_f1']) * 100
qwk_retention = (test_results_student['eval_quadratic_weighted_kappa'] / test_results_teacher['eval_quadratic_weighted_kappa']) * 100

print(f"\n🎯 PERFORMANCE RETENTION:")
print(f"Accuracy Retention: {acc_retention:.1f}%")
print(f"F1-Score Retention: {f1_retention:.1f}%")
print(f"QWK Retention: {qwk_retention:.1f}%")

print(f"\n💡 DISTILLATION SUMMARY:")
print(f"• Temperature: {TEMPERATURE}")
print(f"• Alpha (hard loss weight): {ALPHA}")
print(f"• Achieved {compression_ratio:.1f}x model compression with {speedup:.1f}x inference speedup")
print(f"• Performance drop: {abs(test_results_student['eval_f1'] - test_results_teacher['eval_f1']):.4f} F1-score points")
print(f"• Knowledge retention: {f1_retention:.1f}% of teacher's performance")

# Final wandb log with summary metrics
wandb.log({
    "final_compression_ratio": compression_ratio,
    "final_speedup": speedup,
    "final_f1_retention": f1_retention,
    "final_accuracy_retention": acc_retention,
    "final_qwk_retention": qwk_retention,
    "temperature": TEMPERATURE,
    "alpha": ALPHA
})

print(f"\n✅ Knowledge distillation complete! Results logged to W&B.")

# Save the distilled model
trainer.save_model("./distilled_model_final")
print(f"✅ Distilled model saved to ./distilled_model_final")

# Finish wandb run
wandb.finish()

KNOWLEDGE DISTILLATION RESULTS SUMMARY

📊 MODEL COMPRESSION:
Teacher Model Size: 475.51 MB
Student Model Size: 313.28 MB
Compression Ratio: 1.52x
Size Reduction: 34.1%

⚡ INFERENCE SPEED:
Teacher Model: 15.36 ms
Student Model: 8.66 ms
Speedup: 1.77x

🎯 VALIDATION SET PERFORMANCE:
                      Teacher     Student     Difference
Accuracy:             0.8393      0.8079      -0.0314
F1-Score:             0.8388      0.8058      -0.0330
QWK:                  0.9297      0.9140      -0.0157
MAE:                  0.1820      0.2229      +0.0409

🧪 TEST SET PERFORMANCE:
                      Teacher     Student     Difference
Accuracy:             0.8338      0.8119      -0.0219
F1-Score:             0.8330      0.8098      -0.0232
QWK:                  0.9248      0.9158      -0.0090
MAE:                  0.1930      0.2208      +0.0277

🎯 PERFORMANCE RETENTION:
Accuracy Retention: 97.4%
F1-Score Retention: 97.2%
QWK Retention: 99.0%

💡 DISTILLATION SUMMARY:
• Temperature: 2.0
• Alp

Epoch,▁▂▃▃▄▅▆▆▇█
Learning_Rate,▁▁▁▁▁▁▁▁▁▁
Train Loss,█▅▄▃▃▂▂▁▁▁
Validation Accuracy,▁▂▇▇▄█▆▇▆▇
Validation F1,▁▂▇▇▄█▆▇▆▇
Validation Loss,█▇▁▁▇▂▇▆▇▆
Validation MAE,█▆▂▂▄▁▂▂▂▂
Validation QWK,▁▄▆▇▆███▇█
alpha,▁
eval/accuracy,▁▂▇▇▄▇▆▆▆▆█▆█▇
eval/accuracy_extremely_negative,▁▇▆▆█▇▇▇█▇▇▇▇▇


print()